Training

In [ ]:


import pandas as pd
import os
from PIL import Image

csv_path = r"D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\test_annotations.csv"  # Path to CSV annotations
image_folder = r"D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val"  # Folder with images
output_label_folder = r"D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\labels\val"  # Folder to save YOLO labels

# Ensure label folder exists
os.makedirs(output_label_folder, exist_ok=True)

# Load and sort CSV
df = pd.read_csv(csv_path)
df = df.sort_values(by="filename")

# Save back if needed
df.to_csv(csv_path, index=False)

# Process each annotation
for _, row in df.iterrows():
    filename = row["filename"]
    xmin, xmax, ymin, ymax = row["xmin"], row["xmax"], row["ymin"], row["ymax"]

    # Ensure image exists
    img_path = os.path.join(image_folder, filename)
    if not os.path.exists(img_path):
        continue

    # Load image to get dimensions
    img = Image.open(img_path)
    img_w, img_h = img.size

    # Normalize YOLO format (values between 0 and 1)
    x_center = (xmin + xmax) / (2 * img_w)
    y_center = (ymin + ymax) / (2 * img_h)
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h

    # Always assign class "0" since we are skipping classification
    label_filename = os.path.join(output_label_folder, filename.replace(".jpg", ".txt"))
    with open(label_filename, "w") as f:
        f.write(f"0 {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

print("✅ Annotations converted to YOLO format (Bounding Box Only)!")


In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (pretrained weights)
model = YOLO("yolov8n.pt")  # Using a strong backbone for better detection

# Train only for bounding boxes
model.train(
    data=r"D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\dataset.yaml", # Path to dataset YAML
    epochs=100,
    batch=64,  # Increase batch size if GPU allows
    imgsz=640
)

print("🚀 YOLOv8 Training Complete (Bounding Box Detection Only)!")


Evaluation

In [3]:
import os
import torch
import cv2
import numpy as np
from ultralytics import YOLO
import random

# ======================= Load YOLOv8 Model =======================
model_path = r"D:\Projects\brain_tumor_project\src\components\models\best.pt"  # Path to trained YOLOv8 model
model = YOLO(model_path)

# ======================= Dataset Paths =======================
DATASET_PATH = r"D:\Projects\brain_tumor_project\YOLO_Dataset\dataset"  # Path to dataset
VAL_DIR = os.path.join(DATASET_PATH, "images/val")
VAL_LABELS_DIR = os.path.join(DATASET_PATH, "labels/val")

# ======================= Load Ground Truth Labels (YOLO Format) =======================
def load_ground_truth(label_path, image_width, image_height):
    boxes = []
    with open(label_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # YOLO format: class x_center y_center width height
                _, x_center, y_center, width, height = map(float, parts)
                
                # Convert from normalized (0-1) to absolute pixel values
                x_min = (x_center - width / 2) * image_width
                y_min = (y_center - height / 2) * image_height
                x_max = (x_center + width / 2) * image_width
                y_max = (y_center + height / 2) * image_height
                
                boxes.append([x_min, y_min, x_max, y_max])
    return torch.tensor(boxes)

# ======================= IoU Calculation =======================
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1g, y1g, x2g, y2g = box2
    
    xi1 = max(x1, x1g)
    yi1 = max(y1, y1g)
    xi2 = min(x2, x2g)
    yi2 = min(y2, y2g)
    
    intersection = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    area1 = (x2 - x1) * (y2 - y1)
    area2 = (x2g - x1g) * (y2g - y1g)
    union = area1 + area2 - intersection
    
    return intersection / union if union > 0 else 0

# ======================= Evaluation Metrics =======================
def evaluate_metrics(pred_boxes, gt_boxes, iou_threshold=0.3):
    TP, FP, FN = 0, 0, 0
    
    matched = set()
    
    for pred_box in pred_boxes:
        ious = [calculate_iou(pred_box, gt_box) for gt_box in gt_boxes]
        max_iou = max(ious) if ious else 0
        match_idx = ious.index(max_iou) if max_iou > iou_threshold else -1
        
        if match_idx != -1 and match_idx not in matched:
            TP += 1
            matched.add(match_idx)
        else:
            FP += 1
    
    FN = len(gt_boxes) - len(matched)
    
    precision = TP / (TP + FP + 1e-6)
    recall = TP / (TP + FN + 1e-6)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)
    
    return precision, recall, f1_score

# ======================= Debug Visualization =======================
def visualize_predictions(image_path, pred_boxes, gt_boxes):
    image = cv2.imread(image_path)

    # Draw predicted boxes (Green)
    for box in pred_boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Draw ground truth boxes (Red)
    for box in gt_boxes:
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

    filename = f"/training_new/yolo_outputs/debug_{random.randint(1000,9999)}.jpg"
    cv2.imwrite(filename, image)
#     print(f"🔍 Debug Image Saved: {filename}")

# ======================= Run Evaluation =======================
total_precision, total_recall, total_f1 = 0, 0, 0
num_images = 0

for filename in os.listdir(VAL_DIR):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(VAL_DIR, filename)
        label_path = os.path.join(VAL_LABELS_DIR, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
        
        # Load Image
        image = cv2.imread(image_path)
        image_height, image_width, _ = image.shape

        # Run YOLOv8 Inference
        results = model(image_path)  # Run inference
        pred_boxes = []
        
        for result in results:
            for box in result.boxes.xyxy.cpu().numpy():  # Get bounding boxes
                pred_boxes.append(box)

        # Load Ground Truth
        gt_boxes = load_ground_truth(label_path, image_width, image_height)

        # Evaluate
        precision, recall, f1_score = evaluate_metrics(pred_boxes, gt_boxes)
        total_precision += precision
        total_recall += recall
        total_f1 += f1_score
        num_images += 1

        # Debug Visualization
        visualize_predictions(image_path, pred_boxes, gt_boxes)

# ======================= Print Final Results =======================
mean_precision = total_precision / num_images
mean_recall = total_recall / num_images
mean_f1_score = total_f1 / num_images

print("\n===== FINAL EVALUATION RESULTS (YOLOv8) =====")
print(f"📌 Mean Precision: {mean_precision:.4f}")
print(f"📌 Mean Recall: {mean_recall:.4f}")
print(f"📌 Mean F1-Score: {mean_f1_score:.4f}")
print("✅ Evaluation Completed on Validation Set!")



image 1/1 D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val\1007.jpg: 640x640 1 Tumor, 3082.0ms
Speed: 6.8ms preprocess, 3082.0ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val\1018.jpg: 640x640 1 Tumor, 2512.2ms
Speed: 14.2ms preprocess, 2512.2ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val\1020.jpg: 640x640 1 Tumor, 3438.9ms
Speed: 26.0ms preprocess, 3438.9ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val\1021.jpg: 640x640 1 Tumor, 3032.3ms
Speed: 20.4ms preprocess, 3032.3ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 D:\Projects\brain_tumor_project\YOLO_Dataset\dataset\images\val\103.jpg: 640x640 1 Tumor, 2285.5ms
Speed: 11.7ms preprocess, 2285.5ms inference, 2